In [2]:
#importing the required libraries
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
zip_path="../data/online+retail.zip"
#checking the files in the zip
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # List files in the zip
    print("Files in zip:", zip_file.namelist())


Files in zip: ['Online Retail.xlsx']


In [4]:
# Extract the Excel file
with zipfile.ZipFile("../data/online+retail.zip", "r") as zip_ref:
    zip_ref.extractall('.')
# Load the extracted Excel file
df = pd.read_excel('Online Retail.xlsx')

print(f" Loaded {len(df):,} rows")

 Loaded 541,909 rows


In [5]:
#Inspecting the dataframe
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.shape

(541909, 8)

In [7]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[us]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  str           
dtypes: datetime64[us](1), float64(2), int64(1), object(3), str(1)
memory usage: 33.1+ MB


In [8]:
missing=df.isnull().sum()
print(missing)
missing_pct = (df.isnull().sum() / len(df)) * 100
print(missing_pct)


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64


In [9]:
#fill missing  CustomerID and missing description
df["Description"] = df["Description"].fillna("Unknown Product")
df["CustomerID"]=df["CustomerID"].fillna("Unknown")
print("Missing after cleaning:")
print(df.isnull().sum())

Missing after cleaning:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [10]:
print("BEFORE:")
print(df.dtypes)
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


BEFORE:
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[us]
UnitPrice             float64
CustomerID             object
Country                   str
dtype: object

Memory usage: 141.72 MB


In [11]:
#Converting datatypes
df["InvoiceNo"]=df["InvoiceNo"].astype("category")

df["CustomerID"] = df["CustomerID"].astype("category")

df["Country"] = df["Country"].astype("category")

df["Quantity"] = pd.to_numeric(df["Quantity"], downcast="integer")

df["Description"]=df["Description"].astype("string")

# StockCode to string (has letters)
df["StockCode"] = df["StockCode"].astype("string")


In [12]:
print("AFTER:")
print(df.dtypes)
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


AFTER:
InvoiceNo            category
StockCode              string
Description            string
Quantity                int32
InvoiceDate    datetime64[us]
UnitPrice             float64
CustomerID           category
Country              category
dtype: object

Memory usage: 82.18 MB


In [13]:
#checking wrong dates
invoice_dates = df["InvoiceDate"]
now = pd.Timestamp.now()

print("Date range:")
print(f"Earliest: {invoice_dates.min()}")
print(f"Latest:   {invoice_dates.max()}")

checks = {
    "Future dates (wrong)": (invoice_dates > now),
    "Dates before 2000 (suspicious)": (invoice_dates < "2010-11-01"),
}

for label, condition in checks.items():
    print(f"{label}: {condition.sum()}")
S                -3039.650
D                -5696.220
BANK CHARGES     -7175.639
CRUK             -7933.430
M               -69034.190
AMAZONFEE      -221520.500
Name: Net_Revenue, dtype: float64

Click to add a cell.





SyntaxError: invalid syntax (3045375821.py, line 18)

In [ ]:
mask_dup=df.duplicated()
dup_count=mask_dup.sum()
print(f"Duplicate rows: {dup_count}")

In [ ]:
# See what the duplicates look like
duplicates = df[mask_dup]
print("Sample of duplicate rows:")
print(duplicates.head(10))

# Check which products/invoices have duplicates
print("\nTop duplicated products:")
print(duplicates['StockCode'].value_counts().head(10))

print("\nTop invoices with duplicates:")
print(duplicates['InvoiceNo'].value_counts().head(10))

In [ ]:
# Remove duplicate
df_clean = df.drop_duplicates()
print(
    f"Removed {len(df) - len(df_clean):,} duplicates "
    f"({(len(df) - len(df_clean)) / len(df) * 100:.1f}%)"
)

# Assign back to df to continue cleaning
df = df_clean


In [ ]:
# Quantity check
neg_quantity = df[df['Quantity'] < 0]
zero_quantity = df[df['Quantity'] == 0]

# UnitPrice check
neg_price = df[df['UnitPrice'] < 0]
zero_price = df[df['UnitPrice'] == 0]

print(f"Negative Quantity: {len(neg_quantity)} rows")  # Returns
print(f"Zero Quantity: {len(zero_quantity)} rows")     # Errors
print(f"Negative Price: {len(neg_price)} rows")   # Errors
print(f"Zero Price: {len(zero_price)} rows")      # Free items/errors

In [ ]:
# Keep negative quantity as positive numbers in separate column(Returns quantity and sales quantity)
df["Return_Qty"] = df["Quantity"].where(df["Quantity"] < 0, 0).abs()
df["Sale_Qty"] = df["Quantity"].where(df["Quantity"] > 0, 0)

print(f"Total sold: {df["Sale_Qty"].sum():,}")
print(f"Total returned: {df["Return_Qty"].sum():,}")
df.head(10)

In [ ]:
# Dealing with UnitPrice: separate paid and free items, drop negatives
df = df.copy()

# Drop invalid UnitPrice rows (negative prices)
before = len(df)
df = df[df["UnitPrice"] >= 0]
after = len(df)
print(
    f" Dropped {before - after:,} rows with invalid UnitPrice\n"
    f"   Rows before: {before:,}\n"
    f"   Rows after:  {after:,}"
)

# Create Paid UnitPrice column (numeric for revenue)
df["Paid_UnitPrice"] = df["UnitPrice"].where(df["UnitPrice"] > 0, 0)

# Create Free Item flag column (Yes/No)
df["Is_Free_Item"] = df["UnitPrice"].apply(lambda x: "Yes" if x == 0 else "No")

# Summary counts
paid_count = (df["Paid_UnitPrice"] > 0).sum()
free_count = (df["Is_Free_Item"] == "Yes").sum()

print(f" Paid items: {paid_count:,}")
print(f" Free items: {free_count:,}")


In [ ]:
df.head()

In [ ]:
# Row wise calculation of revenue
df["Revenue"] = df["Sale_Qty"] * df["Paid_UnitPrice"]

# Total revenue
total_revenue = df["Revenue"].sum()
print(f" Total revenue from paid items: ${total_revenue:,.2f}")

In [ ]:
df["Net_Revenue"] = (df["Sale_Qty"] - df["Return_Qty"]) * df["Paid_UnitPrice"]
df.head()

In [ ]:
#total units processed (Sold+returned)
df["Total_Items"] = df["Sale_Qty"] + df["Return_Qty"]
df.head()

In [ ]:
# Aggregate net_revenue by product(accounting for return)
#  - Groups all sales by StockCode (each product)
#  - Sums the Revenue for each product
#  - Sorts in descending order to show top-revenue products first
revenue_per_product = df.groupby("StockCode")["Net_Revenue"].sum().sort_values(ascending=False)
revenue_per_product.tail(20)

In [ ]:
# Aggregate net_revenue by customer(accounting for returns)
#  - Groups all sales by CustomerID
#  - Sums the Revenue for each customer
#  - Sorts in descending order to show top-revenue customers first
revenue_per_customer = df.groupby("CustomerID")["Net_Revenue"].sum().sort_values(ascending=False)
revenue_per_customer

In [ ]:
#  Aggregate net revenue by StockCode
revenue_per_product = df.groupby("StockCode")["Net_Revenue"].sum().sort_values(ascending=False)

#  Define non-product StockCodes (fees, adjustments, single letters)
non_products_list = ["S", "D", "BANK CHARGES", "CRUK", "M", "AMAZONFEE"]

#  Filter out non-products
real_products_revenue = revenue_per_product[~revenue_per_product.index.isin(non_products_list)]

#  inspect top products
print(" Top products by net revenue:")
print(real_products_revenue.head(10))

# inspect non-products separately
non_products_revenue = revenue_per_product[revenue_per_product.index.isin(non_products_list)]
print("\n Non-products / adjustments / fees:")
print(non_products_revenue)


In [ ]:
# Check all rows for DOT to assert it is a real stockcode for a product
df[df['StockCode'] == 'DOT'].head()


In [ ]:
# Profitable products
profitable_products = real_products_revenue[real_products_revenue > 0]

# Products causing losses
loss_products = real_products_revenue[real_products_revenue < 0]

print(f" Number of profitable products: {len(profitable_products)}")
print(f" Number of products with losses: {len(loss_products)}")


In [ ]:

# Top 10 profitable products (largest positive net revenue)
profitable_products = real_products_revenue[real_products_revenue > 0].sort_values(ascending=False)

# Top 10 loss-making products (largest negative net revenue)
loss_products = real_products_revenue[real_products_revenue < 0].sort_values()  
# Display results
print("Top 10 profitable products:")
print(profitable_products.head(10))

print("\n Top 10 leading loss-making products:")
print(loss_products.head(10))


In [ ]:
purchases_per_customer = df.groupby("CustomerID")["InvoiceNo"].nunique()
purchases_per_customer

In [ ]:
# Customer level summary
customer_summary = df.groupby("CustomerID").agg(
    Total_Purchases=("InvoiceNo", "nunique"),   # Number of unique invoices/orders
    Total_Net_Revenue=("Net_Revenue", "sum"),  # Total revenue per customer
    Total_Sale_Qty=("Sale_Qty", "sum"),        # Total quantity sold
    Total_Return_Qty=("Return_Qty", "sum")     # Total returned quantity
)

# calculate average order value (Net Revenue per purchase)
customer_summary["Avg_Order_Value"] = customer_summary["Total_Net_Revenue"] / customer_summary["Total_Purchases"]

# Sort by Total_Net_Revenue descending to see top customers
customer_summary = customer_summary.sort_values(by="Total_Net_Revenue", ascending=False)

# Show top 10 customers
customer_summary.head(10)


In [ ]:
df.head()

In [ ]:
# Calculate Recency (Days since last purchase)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']) 
# most recent purchase
latest_date = df['InvoiceDate'].max()  
# Most recent purchase date for each customer
customer_recency = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()  
 # Calculate days since last purchase
customer_recency['RecencyDays'] = (latest_date - customer_recency['InvoiceDate']).dt.days 
# Merge recency with customer summary
customer_summary = customer_summary.merge(customer_recency[['CustomerID', 'RecencyDays']], on='CustomerID')

# Show the updated customer summary with Recency
customer_summary.head(10)


In [55]:
customer_summary['Return_Rate'] = (
    customer_summary['Total_Return_Qty'] /
    customer_summary['Total_Sale_Qty'].replace(0, np.nan)
)

customer_summary.head()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty,Purchase_Frequency_Monthly,Customer_Value
0,Unknown,3708,1469611.65,479131,209482,396.335396,0,0.437212,269649,111240.00,Positive
1,14646.0,77,279489.02,197491,772,3629.727532,1,0.003909,196719,1155.00,Positive
2,18102.0,62,256438.49,64124,2,4136.104677,0,0.000031,64122,1860.00,Positive
3,17450.0,55,187322.17,69973,964,3405.857636,7,0.013777,69009,206.25,Positive
4,14911.0,248,132458.73,80490,3335,534.107782,0,0.041434,77155,7440.00,Positive


In [50]:
customer_summary['Net_Qty'] = (
    customer_summary['Total_Sale_Qty'] - customer_summary['Total_Return_Qty']
)
customer_summary.head()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty
0,Unknown,3708,1469611.65,479131,209482,396.335396,0,0.437212,269649
1,14646.0,77,279489.02,197491,772,3629.727532,1,0.003909,196719
2,18102.0,62,256438.49,64124,2,4136.104677,0,0.000031,64122
3,17450.0,55,187322.17,69973,964,3405.857636,7,0.013777,69009
4,14911.0,248,132458.73,80490,3335,534.107782,0,0.041434,77155


In [52]:
customer_summary['Purchase_Frequency_Monthly'] = (
    customer_summary['Total_Purchases'] /
    ((customer_summary['RecencyDays'] + 1) / 30)
)
customer_summary.head()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty,Purchase_Frequency_Monthly
0,Unknown,3708,1469611.65,479131,209482,396.335396,0,0.437212,269649,111240.00
1,14646.0,77,279489.02,197491,772,3629.727532,1,0.003909,196719,1155.00
2,18102.0,62,256438.49,64124,2,4136.104677,0,0.000031,64122,1860.00
3,17450.0,55,187322.17,69973,964,3405.857636,7,0.013777,69009,206.25
4,14911.0,248,132458.73,80490,3335,534.107782,0,0.041434,77155,7440.00


In [54]:
customer_summary['Customer_Value'] = np.where(
    customer_summary['Total_Net_Revenue'] > 0,
    'Positive',
    'Negative'
)
customer_summary.tail()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty,Purchase_Frequency_Monthly,Customer_Value
4368,12503.0,1,-1126.00,0,1,-1126.00,337,inf,-1,0.088757,Negative
4369,17603.0,5,-1165.30,74,105,-233.06,49,1.418919,-31,3.000000,Negative
4370,14213.0,1,-1192.20,0,244,-1192.20,371,inf,-244,0.080645,Negative
4371,15369.0,1,-1592.49,0,1,-1592.49,143,inf,-1,0.208333,Negative
4372,17448.0,1,-4287.63,0,1,-4287.63,144,inf,-1,0.206897,Negative


In [46]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return_Qty,Sale_Qty,Paid_UnitPrice,Is_Free_Item,Revenue,Net_Revenue,Total_Items
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,6,2.55,No,15.30,15.30,6
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,6,3.39,No,20.34,20.34,6
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,8,2.75,No,22.00,22.00,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,6,3.39,No,20.34,20.34,6
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,6,3.39,No,20.34,20.34,6


In [56]:
customer_summary.head()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty,Purchase_Frequency_Monthly,Customer_Value
0,Unknown,3708,1469611.65,479131,209482,396.335396,0,0.437212,269649,111240.00,Positive
1,14646.0,77,279489.02,197491,772,3629.727532,1,0.003909,196719,1155.00,Positive
2,18102.0,62,256438.49,64124,2,4136.104677,0,0.000031,64122,1860.00,Positive
3,17450.0,55,187322.17,69973,964,3405.857636,7,0.013777,69009,206.25,Positive
4,14911.0,248,132458.73,80490,3335,534.107782,0,0.041434,77155,7440.00,Positive


In [58]:
known_customers = customer_summary[customer_summary['CustomerID'] != 'Unknown'].copy()
known_customers.head()

,CustomerID,Total_Purchases,Total_Net_Revenue,Total_Sale_Qty,Total_Return_Qty,Avg_Order_Value,RecencyDays,Return_Rate,Net_Qty,Purchase_Frequency_Monthly,Customer_Value
1,14646.0,77,279489.02,197491,772,3629.727532,1,0.003909,196719,1155.00,Positive
2,18102.0,62,256438.49,64124,2,4136.104677,0,0.000031,64122,1860.00,Positive
3,17450.0,55,187322.17,69973,964,3405.857636,7,0.013777,69009,206.25,Positive
4,14911.0,248,132458.73,80490,3335,534.107782,0,0.041434,77155,7440.00,Positive
5,12415.0,26,123725.45,77670,428,4758.671154,23,0.005510,77242,32.50,Positive
